# MySQL connection

Loosely buildt on: https://docs.docker.com/samples/library/mysql/



In [ ]:
%%bash
docker pull mysql:latest

## I want to store the files of the database on the host (my computer, not the container).

I store them in a directory named mysql_databasefiles



In [ ]:
%%bash
# -p flag tells it to "create what is needed" - 
mkdir -p mysql_databasefiles
echo "$(pwd)"

In [ ]:
%%bash
docker run \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "Wooootttt"

### Confirm it runs

In [ ]:
!docker container ls -a

In [ ]:
!ls -l mysql_databasefiles

### Kill it and remove it

In [5]:
%%bash
docker kill my_mysql
docker rm my_mysql
docker container ls -a

my_mysql
my_mysql
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
cc884c1600a3        mongo:latest        "docker-entrypoint.s…"   6 days ago          Up 6 days           0.0.0.0:27017->27017/tcp   mymongo


## Connect to it

In [ ]:
import sys
import mysql.connector

def myconnect(user, pw):
    #try:
        conn = mysql.connector.connect( host='localhost', database='world',user=user, password=pw)
        conn.autocommit = True
        return conn;
    #except Exception as ex:
    #    print(str(ex), file=sys.stderr)
    

rootconn = myconnect('root','deterentysker!42snapsnap')

def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

"Done"    

In [ ]:
sqlQuery("explain world.country")

In [ ]:
sqlQuery("SELECT Name, CountryCode, Population FROM city where population > 8000000")

## Add a new schema

In [ ]:
print ( sqlDo("CREATE DATABASE AWESOMENESS DEFAULT CHARACTER SET 'utf8'"))
"Done"

In [ ]:
print( sqlQuery("show databases"))
"Done"

In [ ]:
print( sqlDo("drop schema if exists AWESOMENESS"))
"Done"

In [ ]:
sqlDo("drop schema if exists AWESOMENESS")

## Explaining a table

In [ ]:
sqlQuery("explain world.country")

In [ ]:
sqlQuery("select Name, GovernmentForm from country where Continent = 'Antarctica'")

# Create a user 

Create a **secret-service** user who
* Can read all world tables
* Alter the GovernmentForm column in the country table


In [ ]:
sqlDo("DROP USER IF EXISTS SecretService")

In [ ]:
sqlDo("CREATE USER 'SecretService' IDENTIFIED BY 'destable&corrupt'")

In [ ]:
sqlQuery("select User, Host from mysql.user")

## Permissions

One can restrict which permissions a user has using this statement:

```sql
GRANT Privilege ON my_database.my_table TO the_user@the_host
```

The following table contains some of the possible values for `Privilege`. The full list is in [the reference manual](https://dev.mysql.com/doc/refman/8.0/en/grant.html).

| Privilege | Meaning and Grantable Levels |
| ----: | :---- | 
| ALL [PRIVILEGES] | Grant all privileges at specified access level except GRANT OPTION and PROXY. |
| ALTER | Enable use of ALTER TABLE. Levels: Global, database, table. |
| CREATE | Enable database and table creation. Levels: Global, database, table. |
| CREATE USER | Enable use of CREATE USER, DROP USER, RENAME USER, and REVOKE ALL PRIVILEGES. Level: Global. |
| DELETE | Enable use of DELETE. Level: Global, database, table. |
| DROP | Enable databases, tables, and views to be dropped. Levels: Global, database, table. |
| GRANT OPTION | Enable privileges to be granted to or removed from other accounts. Levels: Global, database, table, routine, proxy. |
| INSERT | Enable use of INSERT. Levels: Global, database, table, column. |
| SELECT | Enable use of SELECT. Levels: Global, database, table, column. |
| SHOW DATABASES | Enable SHOW DATABASES to show all databases. Level: Global. |
| UPDATE | Enable use of UPDATE. Levels: Global, database, table, column. |
| USAGE | Synonym for “no privileges” |![image.png](attachment:image.png)


In [ ]:
sqlQuery("SHOW GRANTS FOR SecretService")

In [ ]:
secretconn = myconnect('SecretService','destable&corrupt')

In [ ]:
sqlDo("GRANT SELECT ON world.* TO SecretService")
sqlDo("FLUSH Privileges")

In [ ]:
sqlQuery("Select Code, Name from country where Continent='Africa' and Population<500000", secretconn)

In [ ]:
sqlQuery("Select * from city where CountryCode='ESH'", secretconn)

In [ ]:
sqlDo("DELETE FROM city WHERE Name = 'DessertHideout'")

In [ ]:
sqlDo("""
INSERT INTO city (Name, CountryCode,District,Population) 
    VALUES ('DessertHideout', 'ESH', 'Sandworms beware', 1200)
""",secretconn)

In [ ]:
sqlDo("GRANT UPDATE, INSERT  ON world.city TO SecretService; flush privileges")

## Create one user more

Create censusSurveyer (person who counts inhabitants). 

* This person should be able to read the world database
* **Update the population columns of city and courntry, but nothing else**

In [ ]:
sqlDo("DROP USER IF EXISTS CensusSurveyer")

In [ ]:
sqlDo("CREATE USER 'CensusSurveyer' IDENTIFIED BY 'count#66behappy'")

In [ ]:
sqlDo("GRANT SELECT ON world.* TO CensusSurveyer")
sqlDo("FLUSH Privileges")

In [ ]:
sqlQuery("select User, Host from mysql.user")

In [ ]:
sqlQuery("SHOW GRANTS FOR CensusSurveyer")

In [ ]:
censusconn = myconnect('CensusSurveyer','count#66behappy')

In [ ]:
query = """
SELECT * 
FROM city 
ORDER BY Population DESC
LIMIT 10;
"""
sqlQuery( query,  censusconn)

In [ ]:
sqlDo("GRANT UPDATE (Population) ON world.city TO CensusSurveyer");
sqlDo("FLUSH Privileges")

In [ ]:
sqlDo("UPDATE city SET Population = Population + 100 WHERE ID=1024",censusconn)

In [ ]:
sqlDo("UPDATE country SET Population = Population + 1000000 WHERE Code='IND'", censusconn)

In [ ]:
sqlDo("UPDATE city SET Name = 'Москва' WHERE ID=3580",censusconn)

### Limiting actions by host

It is possible to grant different permissions depending on from which machine a user is logged in.

There are (at least) three useful scenarios for this:

* Restricting admin users (and in particularly root) to 'localhost'
* Restricting other users to 'localhost', so only users running on the same machine as the DB can access the DB
* Restricting users to a specific set of hosts - so only access from those hosts are valid. Think: DB on one machine and webapps on an other.

## Logs
They serve several roles in serious database maintenance
* Documents all log-ins, and from where
* Documents all queries, for debugging purposes
* Documents performance issues

Besides, the logs play a role in hard recovery of crashed databases

In [ ]:
sqlDo("SET global general_log = 1; SET global log_output = 'table'")

In [ ]:
sqlQuery("Explain mysql.general_log")

In [ ]:
sqlQuery("""
SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host, command_type, argument 
FROM mysql.general_log
ORDER BY time DESC
LIMIT 10
""")

In [ ]:
sqlQuery("""
SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host,  command_type, argument 
FROM mysql.general_log
WHERE command_type <>'Query'
ORDER BY time DESC
LIMIT 15
""")

In [ ]:
sqlQuery("show tables in mysql")

In [ ]:
sqlQuery("show schemas")

## Backup & restore

The final aspect of security is to be able to recover from a break-down or a hack.

The literature mentions a couple of good concepts which are worth noticing:

* **Physical vs. Logical backup**. Do you backup "all the files of the DB" (Physical), or do you create a (number of) sql files which can be run to restore the database (Logical).
* **Online Versus Offline Backups**. Online backups take place while the MySQL server is running so that the database information can be obtained from the server. Offline backups take place while the server is stopped.
* **Local Versus Remote Backups**. A local backup is performed on the same host where the MySQL server runs, whereas a remote backup is done from a different host.
* **Full Versus Incremental Backups**. A full backup includes all data managed by a MySQL server at a given point in time. An incremental backup consists of the changes made to the data during a given time span.


### full local online logical backup

mysql has a special program named `mysqldump` which can do several of the combinations mentioned above. 

The idea is to backup as: `bash> mysqldump --all-databases > dump.sql`.

Restore can then be done as: `bash> mysql < dump.sql`.



### Storing the backup outside the docker container
I want the backup to end up on my host, not inside the filesystem of the docker container.

We are free to chose where to put it. Inside the container I will store to a directory `/mnt/host`, and map that to `$(pwd)/mysql_dump`. 


In [6]:
%%bash
docker kill my_mysql
docker rm my_mysql
docker container ls -a

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
cc884c1600a3        mongo:latest        "docker-entrypoint.s…"   6 days ago          Up 6 days           0.0.0.0:27017->27017/tcp   mymongo


Error response from daemon: Cannot kill container: my_mysql: No such container: my_mysql
Error: No such container: my_mysql


In [9]:
%%bash
docker run \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-v $(pwd)/mysql_dump:/mnt/host \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "Back online"

Back online


docker: Error response from daemon: Conflict. The container name "/my_mysql" is already in use by container "d2eac72dc57a4d82e76b5e37c83e316dd019a4d3494a55c8f4654c677a986c99". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [12]:
%%bash
docker run \
--name my_mysql2 \
-v $(pwd)/mysql_databasefiles2:/var/lib/mysql \
-v $(pwd)/mysql_dump:/mnt/host \
-e MYSQL_ROOT_PASSWORD=bla_open_now \
-d \
mysql
echo "Back online"

2ab34d349f6c409c2246beb3767a977ff0058f5e45f027a6a10a6703e286fc8c
Back online
